# Regression analysis consists of a set of machine learning methods that allow us to predict a continuous outcome variable (y) based on the value of one or multiple predictor variables (x). ... Linear regression is the most simple and popular technique for predicting a continuous variable.
http://www.sthda.com/english/wiki/regression-analysis-essentials-for-machine-learning

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Supervised learning Regression
if the desired output consists of one or more continuous variables, then the task is called regression. An example of a regression problem would be the prediction of the length of a salmon as a function of its age and weight. 

Found from https://scikit-learn.org/stable/tutorial/basic/tutorial.html

Note: y_val == y_validation == y_test

Note: X_val == X_validation == X_test

In [ ]:
import xgboost as xgb
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
train.shape,test.shape

# We can understand that all the freq are relatively the same

In [ ]:
#train.target.hist()

# We can tell that further down the line id must be dropped

In [ ]:
train.describe()

# No nans
# # Aswell as need to drop id as its a useless column

In [ ]:
train.isna().sum() 

In [ ]:
test.head()

# now we can focus on the continous variable columns

# Judgin by top to bottom, we can see that cont variables 1,6,7,8,9,10,11,12,13 are the most correlated.
This must have some purpose or meaning 

In [ ]:
train.corr(method = "pearson").style.background_gradient(cmap='Reds')

prepare for training

In [ ]:
train.shape

In [ ]:
test.shape

# Lets first try with LGBM regressor by itself

In [ ]:
features = ['cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont9','cont10','cont11','cont12','cont13','cont14']
#features = ['cont1','cont6','cont7','cont8','cont9','cont10','cont11','cont12','cont13']
y = train['target']

X_train,X_test, y_train,y_test = train_test_split(train[features],train['target'],test_size=0.15)

#Shape wise doesnt matter until later, but we do want a more larger training size
#85/15  train/test split
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
from lightgbm import LGBMRegressor
import lightgbm as lgbm
#note a booster needs to be specified, dart wont work
param = {'boosting_type': 'gbdt',
         'objective': "regression", 'metric': "RMSE",
         'learning_rate': 0.0045,
        }


model = LGBMRegressor(**param)

predict() returns an integer array 

Also note that when using .fit() the input array lengths must both be the same. In this case its 200000

This was the goal from the beginning, predicting against the 200000 test set.

In [ ]:
model.fit(X_train, y_train)

# Used for combining data for submission
ypred = model.predict(test[features])

#
ypred2 = model.predict(X_test)

# As we can see below doing simple lgbm regression is Alright. The result was a score that is really bad.

In [ ]:
#check what result we got for rmse
#squared = FALSE , If True returns MSE value, if False returns RMSE value.

import math
#rmse always takes in validation sets, eg. y test, x test predicted.
rmse = np.sqrt(mean_squared_error(y_test, ypred2))
rmse

In [ ]:
sub['target'] = ypred 
sub.to_csv("submission.csv",index=False)
sub.head()
print("success!")

# Why dont we try k-fold with lgbm?


# 1. Create kfold, shuffle true and number of splits being done is 3

 Shuffle the dataset randomly. Also we need our target variable.

In [ ]:
#remember to not define y twice
y = train['target']

# 2. Split the dataset into k groups

In [ ]:
kf = KFold(n_splits=3, shuffle=True, random_state=1)

# 3.Create params, note that most params will be used for accuracy. 

Metrics will be RMSE 

boosting type will be gbdt used instead of dart. Dart does not allow early stopping

Use small learning_rate with large num_iterations. This way you can get the model to learn more overtime with more large amount of data. Note that we cant specify num_iterations as this conflicts with the boost.

In [ ]:
params = {   "metric": "rmse",
              "boosting_type": "gbdt",
              "learning_rate":0.01,
        }

# 4. Split data and fit data

# code structure referenced from https://www.kaggle.com/fatihozturk/lgbm-model-initialisation-trick

In [ ]:
import lightgbm as lgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


#split the training data
for train_split, test_split in kf.split(train[features], train['target']):
    
    #acquire the train and test 
    X_train= train.iloc[train_split]
    X_test = train.iloc[test_split]
    y_train = y.iloc[train_split]
    y_test = y.iloc[test_split]
    
    d_train = lgb.Dataset(X_train[features], label= y_train)
    d_test = lgb.Dataset(X_test[features], label = y_test)
    
    #If you want early stopping, you need to provide validation set, 
    #as the error message clearly states. And you need to do it in a fit method
    
    #If verbose_eval is True then the evaluation metric on the 
    #validation set is printed at each boosting stage
    
    #num_boost_round = Number of boosting iterations.
    model = lgbm.train(params,d_train,valid_sets=[d_train, d_test],verbose_eval=100,num_boost_round=1000,early_stopping_rounds=100)
    
    #Prediction
    y_pred=model.predict(X_test[features])
    
    #For each rmse score obtained, add to the score_storage list 
    score = np.sqrt(mean_squared_error(y_test, y_pred))